In [0]:
WITH bounds AS (
    SELECT
        MIN(ingest_time) AS earliest_ingest,
        MAX(ingest_time) AS latest_ingest
    FROM edsm.silver.v_powerplay
),
changing_ids AS (
    SELECT id, power
    FROM edsm.silver.v_powerplay
    WHERE ingest_time BETWEEN (SELECT earliest_ingest FROM bounds) AND (SELECT latest_ingest FROM bounds)
    GROUP BY id, power
    HAVING COUNT(DISTINCT struct(allegiance, government, powerState, state)) > 1
)
SELECT
    valid_from,
    valid_to,
    name,
    power,
    allegiance,
    government,
    powerState,
    state
FROM edsm.silver.v_powerplay
WHERE (id, power) IN (SELECT id, power FROM changing_ids)
and name = "Styx"
AND ingest_time BETWEEN (SELECT earliest_ingest FROM bounds) AND (SELECT latest_ingest FROM bounds)
ORDER BY id, power, valid_from;

In [0]:
%python

dbutils.widgets.text("system_name", "")
dbutils.widgets.text("power", "")

system_name = dbutils.widgets.get("system_name")
power = dbutils.widgets.get("power")

latest_ingest = spark.sql("SELECT MAX(ingest_time) AS latest FROM edsm.silver.v_powerplay").collect()[0]["latest"]

query = f"""
SELECT
    name,
    power,
    powerState,
    state,
    valid_from,
    CASE
        WHEN valid_to > TIMESTAMP('{latest_ingest}') THEN TIMESTAMP('{latest_ingest}')
        ELSE valid_to
    END AS valid_to
FROM edsm.silver.v_powerplay
WHERE name = '{system_name}' AND power = '{power}'
ORDER BY valid_from
"""

df = spark.sql(query).toPandas()

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

df['valid_from'] = pd.to_datetime(df['valid_from'])
df['valid_to'] = pd.to_datetime(df['valid_to'])

fig, ax = plt.subplots(figsize=(12, len(df) * 0.4))

for i, row in df.iterrows():
    ax.barh(
        y=i,
        width=(row['valid_to'] - row['valid_from']).days,
        left=row['valid_from'],
        height=0.9
    )

ax.set_yticks(range(len(df)))
ax.set_yticklabels([f"{row['powerState']} - {row['state']}" for _, row in df.iterrows()])
ax.invert_yaxis()
ax.set_xlabel("Date")
ax.set_title(f"{system_name} - {power}")

ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
fig.autofmt_xdate(rotation=0)

plt.tight_layout()
plt.show()